In [54]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [55]:
import requests
from bs4 import BeautifulSoup
import re

# ニュース一覧ページのURL
url = "https://news.yahoo.co.jp/"

# ページのHTMLを取得
response = requests.get(url)
response.encoding = response.apparent_encoding  # 文字コードを自動判別して設定

# HTMLをBeautifulSoupでパースし、body部分を取り出す
soup = BeautifulSoup(response.text, "html.parser")
body_html = str(soup.body)  # body部分のHTMLを文字列として取得
# ニュース一覧の情報を抽出
news_list = soup.find_all("li", class_="sc-1nhdoj2-0 gXbfsN")
pickup_link_list = []
# ニュース情報の取得
for news in news_list:
    
    # ニュースのリンクを取得
    link = news.find("a", class_="sc-1nhdoj2-1 bfNCyS")["href"]
    pickup_link_list.append(link)
    
    print(f"Link: {link}")
    print("-" * 40)

Link: https://news.yahoo.co.jp/pickup/6571060
----------------------------------------
Link: https://news.yahoo.co.jp/pickup/6571058
----------------------------------------
Link: https://news.yahoo.co.jp/pickup/6571055
----------------------------------------
Link: https://news.yahoo.co.jp/pickup/6571051
----------------------------------------
Link: https://news.yahoo.co.jp/pickup/6571052
----------------------------------------
Link: https://news.yahoo.co.jp/pickup/6571064
----------------------------------------
Link: https://news.yahoo.co.jp/pickup/6571059
----------------------------------------


In [56]:
news_text = []
for link in pickup_link_list:
    print("link: ", link)
    # ページのHTMLを取得
    response = requests.get(link)
    response.encoding = response.apparent_encoding  # 文字コードを自動判別して設定

    # HTMLをBeautifulSoupでパースし、body部分を取り出す
    soup = BeautifulSoup(response.text, "html.parser")
    body_html = str(soup.body)  # body部分のHTMLを文字列として取得
    # ニュース一覧の情報を抽出
    try:
        url = soup.find("a", class_="sc-1gj6noq-9 cbHnkd")["href"]
    except TypeError:
        continue
    # ページのHTMLを取得
    article_response = requests.get(url)
    article_response.encoding = article_response.apparent_encoding  # 文字コードを自動判別して設定

    # HTMLをBeautifulSoupでパースし、body部分を取り出す
    article_soup = BeautifulSoup(article_response.text, "html.parser")
    article_body_html = str(article_soup.body)  # body部分のHTMLを文字列として取得
    print("body: ", article_body_html[15000:20000]) # 結果を表示して確認
    news_text.append({"url": url, "body": article_body_html})
    break

link:  https://news.yahoo.co.jp/pickup/6571060
link:  https://news.yahoo.co.jp/pickup/6571058
body:  d riff-ClickableState__root riff-z-10 riff-absolute riff-top-0 riff-right-0 riff-bottom-0 riff-left-0 riff-rounded-tl-inherit riff-rounded-tr-inherit riff-rounded-bl-inherit riff-rounded-br-inherit riff-opacity-0 riff-pointer-events-none"></div><div class="riff-Clickable__focus riff-ClickableFocus__root--hideFocusRing riff-ClickableFocus__root--outside riff-ClickableState__root riff-z-10 riff-absolute riff-top-0 riff-right-0 riff-bottom-0 riff-left-0 riff-rounded-tl-inherit riff-rounded-tr-inherit riff-rounded-bl-inherit riff-rounded-br-inherit riff-opacity-0 riff-pointer-events-none"></div></a></div></div></div></div><a class="sc-1enscux-0 ipFzZV" data-cl-params="_cl_vmodule:detail;_cl_link:banner;" href="https://www.mbs.jp/news/"><img alt="MBSニュース" height="40" src="https://s.yimg.jp/images/news/cobranding/mbsnews.png" width="170"/></a></div></header><div class="article_body highLightS

In [57]:
# LLMにニュースの要約を抽出させるプロンプトを作成
prompt = f"""
以下のHTMLからニュースの内容を要約して出力してください。

#HTML:
{news_text[0]["body"]}
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)

# LLMからの回答を表示
print("link: ", news_text[0]["url"])
print("summary: ", response.choices[0].message.content.strip())

link:  https://news.yahoo.co.jp/articles/9143e779988f16b5c3aca69866d1fd769e4f2450
summary:  滋賀県大津市で、36歳の男が担当保護司を面接中に斧やナイフで殺害した事件について、検察側が「無期懲役」を求刑しました。この事件は2024年5月に発生し、男の刑事責任能力が争点となっています。初公判で被告は「守護神様の声に従ってやりました」と述べ、弁護側は責任能力の低下を主張しています。

検察は、被告が自暴自棄になり、保護観察制度に対する報復を考えていたと指摘。犯行当日は新庄博志さん（当時60歳）を背後から襲い、制止の声を無視して攻撃を続けました。被告は保護観察中の面談を「茶番」と感じていたと証言しており、国の問題改善を促すために事件を起こしたとの考えも示しています。
